# Classificação de Texto e Reranqueadores

Desenvolvimento de um buscador Simples: Booleano, TF-IDF, BM25

Tópicos abordados: Indexação, Bag-of-Words, TF-IDF, BM25

Aula 1 - [Unicamp - IA368DD: Deep Learning aplicado a sistemas de busca.](https://www.cpg.feec.unicamp.br/cpg/lista/caderno_horario_show.php?id=1779)

Autor: Marcus Vinícius Borela de Castro

[Repositório no github](https://github.com/marcusborela/deep_learning_em_buscas_unicamp)

[Link para chat de apoio com WebChatGPT](https://github.com/marcusborela/deep_learning_em_buscas_unicamp/blob/main/chat/CG%20uso%20no%20buscador%20aula%201.md)

[![Open In Colab latest github version](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/marcusborela/deep_learning_em_buscas_unicamp/blob/main/code/aula_2_classificacao_de_texto_e_reranqueador.ipynb) [Open In Colab latest github version]

# Enunciado exercício

Aula 2 - Notebook: Classificação de Texto e Reranqueadores

Data de entrega: 15 de mar., 23:59

Reranqueamento usando um modelo estilo-BERT com o treinamento no dataset do MS MARCO e avaliação no TREC-DL 2020


O treinamento é igual ao de um classificador binário, que será feito por vocês.


O que muda é a forma de avaliação: reranqueadores precisam ser alimentados com documentos candidatos (ex: trazidos pelo BM25 - exercício aula 1)


Sugestões:
1. usar este dataset reduzido do MS MARCO como treinamento, com 10k triplas (query, passagem relevante, passagem não-relevante):
https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv


2. usar miniLM (modelo BERT pequeno, 5x mais rapido) para começar o finetuning: https://huggingface.co/nreimers/MiniLM-L6-H384-uncased pois oferece um bom compromisso entre qualidade e velocidade.


3. usar este notebook como base
Análise de sentimentos (dataset IMDB) usando um modelo estilo BERT: https://colab.research.google.com/drive/10etP7Lb915EC-uEuf1IKC8DYkyg_om6-?usp=sharing


4. para debug: usar este minilm para ver se consegue ndcg ~0.70: https://huggingface.co/cross-encoder/ms-marco-MiniLM-L-6-v2Sugestão: fazer overfit em um batch: treinar por 200 epocas um unico batch, e ver se consegue loss=0, e acuracia=100%, ou ndcg=1



# Organizando o ambiente

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Mar 11 17:51:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    29W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory


In [3]:
def mostra_memoria():
  vm = virtual_memory()
  ram={}
  ram['total']=round(vm.total / 1e9,2)
  ram['available']=round(virtual_memory().available / 1e9,2)
  # ram['percent']=round(virtual_memory().percent / 1e9,2)
  ram['used']=round(virtual_memory().used / 1e9,2)
  ram['free']=round(virtual_memory().free / 1e9,2)
  ram['active']=round(virtual_memory().active / 1e9,2)
  ram['inactive']=round(virtual_memory().inactive / 1e9,2)
  ram['buffers']=round(virtual_memory().buffers / 1e9,2)
  ram['cached']=round(virtual_memory().cached/1e9 ,2)
  print(f"Your runtime RAM in gb: \n total {ram['total']}\n available {ram['available']}\n used {ram['used']}\n free {ram['free']}\n cached {ram['cached']}\n buffers {ram['buffers']}")


In [4]:
mostra_memoria()

Your runtime RAM in gb: 
 total 27.33
 available 26.27
 used 0.7
 free 23.25
 cached 3.1
 buffers 0.28


### Vinculando pasta do google drive para salvar dados

In [5]:
import os

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
current_dir = os.getcwd()
print("Current directory:", current_dir)

Current directory: /content


### Instalações de libraries

In [8]:
!pip install git+https://github.com/castorini/pygaggle.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/castorini/pygaggle.git to /tmp/pip-req-build-iap_1v0w
  Running command git clone --filter=blob:none --quiet https://github.com/castorini/pygaggle.git /tmp/pip-req-build-iap_1v0w
  Resolved https://github.com/castorini/pygaggle.git to commit c285f6084684367dd07b608ef19c2722b5b0637e
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 KB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.1/137.1 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 97.7 MB/s eta 0:00:00
     

In [9]:
!pip install pyserini

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
!pip install faiss-cpu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 73.7 MB/s eta 0:00:00


### Baixando o repositório do pyserini para usara seus scripts

In [10]:
path_pyserini = '/content/drive/MyDrive/treinamento/202301_IA368DD/code/pyserini'
path_pyserini_tools = path_pyserini + '/pyserini-master/anserini-tools-master'
path_pyserini_eval = path_pyserini + '/pyserini-master/pyserini/eval'

In [14]:
if not os.path.exists(path_pyserini):
    os.makedirs(path_pyserini)
    print('pasta criada')
    !wget -q https://github.com/castorini/pyserini/archive/refs/heads/master.zip -O pyserini.zip 
    !unzip -q pyserini.zip -d  {path_pyserini}
    # Baixando tools que é um atalho para https://github.com/castorini/anserini-tools
    !wget -q https://github.com/castorini/anserini-tools/archive/refs/heads/master.zip -O anserini-tools.zip 
    !unzip -q anserini-tools.zip -d  {path_pyserini}


In [ ]:
path_pyserini = path_pyserini + '/pyserini-master'

In [12]:
 assert os.path.exists(path_pyserini), f"Pasta {path_pyserini} não criada!"

In [15]:
 assert os.path.exists(path_pyserini_tools), f"Pasta {path_pyserini_tools} não criada!"

In [16]:
 assert os.path.exists(path_pyserini_eval), f"Pasta {path_pyserini_eval} não criada!"

# Carga dos dados da TREC 2020 usando pyserini

### Obtendo dados dos documentos a partir do pyserini


[Dicas aqui](https://github.com/castorini/pyserini/blob/master/docs/experiments-msmarco-passage.md)

In [17]:
path_data = '/content/drive/MyDrive/treinamento/202301_IA368DD/collections/msmarco-passage'

In [18]:
%%time
if not os.path.exists(path_data):
  os.makedirs(path_data)
  print('pasta criada')
  !wget https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz -P {path_data}
  !tar xvfz {path_data}/collectionandqueries.tar.gz -C {path_data}
  os.remove(f'{path_data}/collectionandqueries.tar.gz')
  print("Dados carregados!")
else:
  print("Dados já existiam!")    

Dados já existiam!
CPU times: user 1.17 ms, sys: 0 ns, total: 1.17 ms
Wall time: 1.02 ms


In [19]:
assert os.path.exists(path_data), f"Pasta {path_data} não criada!"

Passo anterior gera os seguintes arquivos:

* collection.tsv
* qrels.dev.small.tsv
* qrels.train.tsv
* queries.dev.small.tsv
* queries.dev.tsv
* queries.eval.small.tsv
* queries.eval.tsv
* queries.train.tsv

Next, we need to convert the MS MARCO tsv collection into Pyserini's jsonl files (which have one json object per line):

In [21]:
%%time
if not os.path.exists(f'{path_data}/collection_jsonl'):
  !python {path_pyserini_tools}/tools/scripts/msmarco/convert_collection_to_jsonl.py \
  --collection-path {path_data}/collection.tsv \
  --output-folder {path_data}/collection_jsonl
  print("Dados carregados!")
else:
  print("Dados já existiam!")    

Dados já existiam!
CPU times: user 1.14 ms, sys: 4 µs, total: 1.15 ms
Wall time: 896 µs


In [20]:
assert os.path.exists(f'{path_data}/collection_jsonl'), f"Pasta {path_data}/collection_jsonl não criada!"

The above script should generate 9 jsonl files in collections/msmarco-passage/collection_jsonl, each with 1M lines (except for the last one, which should have 841,823 lines).

### Loading data in dicts

The 6980 queries in the development set are already stored in the repo. Let's take a peek:

In [22]:
!head {path_pyserini_tools}/topics-and-qrels/topics.msmarco-passage.dev-subset.txt

1048585	what is paula deen's brother
2	 Androgen receptor define
524332	treating tension headaches without medication
1048642	what is paranoid sc
524447	treatment of varicose veins in legs
786674	what is prime rate in canada
1048876	who plays young dr mallard on ncis
1048917	what is operating system misconfiguration
786786	what is priority pass
524699	tricare service number


In [23]:
!head {path_pyserini_tools}/topics-and-qrels/topics.dl20.txt

1030303	who is aziz hashim
1037496	who is rep scalise?
1043135	who killed nicholas ii of russia
1045109	who owns barnhart crane
1049519	who said no one can make you feel inferior
1051399	who sings monk theme song
1056416	who was the highest career passer  rating in the nfl
1064670	why do hunters pattern their shotguns?
1065636	why do some places on my scalp feel sore
1071750	why is pete rose banned from hall of fame


In [24]:
!head {path_data}/qrels.dev.small.trec

300674 0 7067032 1
125705 0 7067056 1
94798 0 7067181 1
9083 0 7067274 1
174249 0 7067348 1
320792 0 7067677 1
1090270 0 7067796 1
1101279 0 7067891 1
201376 0 7068066 1
54544 0 7068203 1


In [25]:
!head {path_pyserini_tools}/topics-and-qrels/qrels.dl20-passage.txt

23849 0 1020327 2
23849 0 1034183 3
23849 0 1120730 0
23849 0 1139571 1
23849 0 1143724 0
23849 0 1147202 0
23849 0 1150311 0
23849 0 1158886 2
23849 0 1175024 1
23849 0 1201385 0


Each line contains a tab-delimited (query id, query) pair. Conveniently, Pyserini already knows how to load and iterate through these pairs. We can now perform retrieval using these queries:

#### Carregando qrel (relevância por query)

##### Carregando usando get_qrels

In [26]:
from pyserini.search import get_qrels

In [27]:
qrels = get_qrels('dl20-passage')

In [28]:
len(qrels)

54

In [29]:
list(qrels.items())[0]

(23849,
 {1020327: '2',
  1034183: '3',
  1120730: '0',
  1139571: '1',
  1143724: '0',
  1147202: '0',
  1150311: '0',
  1158886: '2',
  1175024: '1',
  1201385: '0',
  1215556: '0',
  1220759: '0',
  1221770: '0',
  1333480: '1',
  1381453: '2',
  1414114: '2',
  1414115: '0',
  1414120: '2',
  1449780: '0',
  146754: '0',
  1493231: '0',
  1532701: '0',
  1535484: '0',
  1605854: '1',
  1605857: '1',
  1622747: '1',
  17118: '0',
  17122: '0',
  1714915: '0',
  1714917: '1',
  1724687: '0',
  172488: '0',
  178252: '0',
  182049: '0',
  1827512: '1',
  1844627: '0',
  188190: '0',
  188246: '1',
  1944730: '0',
  2003292: '0',
  2017213: '0',
  2203364: '0',
  2209883: '0',
  2318793: '0',
  2339898: '1',
  2373852: '0',
  2397072: '0',
  2423771: '0',
  2516458: '0',
  2585563: '0',
  2593928: '0',
  2607127: '3',
  2607128: '1',
  2607129: '2',
  2607130: '2',
  2607131: '2',
  2607132: '3',
  2607134: '0',
  2647769: '3',
  2674124: '0',
  2766280: '0',
  282421: '0',
  2838462: 

#### Carregando queries

##### Carregando usando get_topics

In [30]:
from pyserini.search import get_topics

In [31]:
topics = get_topics('dl20')
print(f'{len(topics)} queries total')

200 queries total


In [32]:
len(topics), list(topics.items())[0]

(200, (735922, {'title': 'what is crimp oil'}))

Deixando apenas queries com relevância

In [33]:
topics = {key:value for key, value in topics.items() if key in qrels}

In [34]:
len(topics), list(topics.items())[0]

(54, (23849, {'title': 'are naturalization records public information'}))

# Executando 1o estágio: exact match com BM25

In [35]:
num_max_hits = 1000

#### Com código para retornar o score do bm25

In [36]:
# Run all queries in topics, retrive top 1k for each query
def run_all_queries(file, topics, searcher, num_max_hits=100):
    print(f'Running {len(topics)} queries in total')
    with open(file, 'w') as runfile:
        cnt = 0
        for id in tqdm(topics, desc='Running Queries'):
            query = topics[id]['title']
            hits = searcher.search(query, num_max_hits)
            for i in range(0, len(hits)):
                _ = runfile.write(f'{id} Q0 {hits[i].docid} {i+1} {hits[i].score:.6f} Etapa_EM\n')
                # = runfile.write('{} Q0 {} {} {:.6f} Pyserini\n'.format(id, hits[i].docid, i+1, hits[i].score))
            cnt += 1
            if cnt % 100 == 0:
                print(f'{cnt} queries completed')

In [37]:
path_run = path_data + '/runs'
path_run_estagio1 = path_run + '/run-msmarco-passage-bm25-estagio1.txt'

In [38]:
path_run, path_run_estagio1

('/content/drive/MyDrive/treinamento/202301_IA368DD/collections/msmarco-passage/runs',
 '/content/drive/MyDrive/treinamento/202301_IA368DD/collections/msmarco-passage/runs/run-msmarco-passage-bm25-estagio1.txt')

In [39]:
from tqdm import tqdm

In [41]:
%%time
if not os.path.exists(path_run):
  os.makedirs(path_run)
  print('pasta criada!')
else:
  print('pasta já existia!')


CPU times: user 1.04 ms, sys: 0 ns, total: 1.04 ms
Wall time: 1.49 ms


In [42]:
if not os.path.exists(path_run_estagio1):
  # roda 1o estágio de busca por bm25
  # code from https://colab.research.google.com/github/castorini/anserini-notebooks/blob/master/pyserini_msmarco_passage_demo.ipynb
  from pyserini.search import SimpleSearcher
  from pyserini.search.lucene import LuceneSearcher


  # Indexando Trec 2020 Collection using Pyserini
  !python -m pyserini.index.lucene \
    --collection JsonCollection \
    --input {path_data}/collection_jsonl \
    --index indexes/lucene-index-msmarco-passage \
    --generator DefaultLuceneDocumentGenerator \
    --threads 9 \
    --storePositions --storeDocvectors --storeRaw

  # realizando busca e salvando dados do estágio 1
  searcher = LuceneSearcher('./indexes/lucene-index-msmarco-passage')
  searcher.set_bm25(k1=0.82, b=0.68)  
  run_all_queries(path_run_estagio1, topics, searcher, num_max_hits)
  
  print("Dados estágio 1 (bm25) carregados!")
else:
  print("Dados estágio 1 (bm25) já existiam!")    

Dados estágio 1 (bm25) já existiam!


In [40]:
assert os.path.exists(path_run_estagio1), f"Pasta {path_run_estagio1} não criada!"

In [43]:
!head {path_run_estagio1}

23849 Q0 4348282 1 10.738400 Etapa_EM
23849 Q0 2674124 2 10.426600 Etapa_EM
23849 Q0 542113 3 9.687200 Etapa_EM
23849 Q0 8246990 4 9.641500 Etapa_EM
23849 Q0 8133127 5 9.566500 Etapa_EM
23849 Q0 2647769 6 9.547200 Etapa_EM
23849 Q0 2017213 7 9.504800 Etapa_EM
23849 Q0 7119957 8 9.500700 Etapa_EM
23849 Q0 964148 9 9.476800 Etapa_EM
23849 Q0 436721 10 9.464000 Etapa_EM


##### Eval

In [45]:
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 -mrecall.1000 -mmap -l 2 'dl20-passage' {path_run_estagio1}

jtreceval-0.0.5-jar-with-dependencies.jar: 1.79MB [00:01, 946kB/s]                 
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', '-mrecall.1000', '-mmap', '-l', '2', '/root/.cache/pyserini/topics-and-qrels/qrels.dl20-passage.txt', '/content/drive/MyDrive/treinamento/202301_IA368DD/collections/msmarco-passage/runs/run-msmarco-passage-bm25-estagio1.txt']
Results:
map                   	all	0.2876
recall_1000           	all	0.8031
ndcg_cut_10           	all	0.4876


# Treinamento do ranqueador

Fonte: https://colab.research.google.com/drive/10etP7Lb915EC-uEuf1IKC8DYkyg_om6-?usp=sharing

In this notebook, we will finetuned and evaluate a BERT-based model on a document classification task.

Our dataset will be a smaller version of the IMDB Sentiment Analysis dataset.

To finetune and evaluate faster, we will be using [miniLM](https://huggingface.co/microsoft/MiniLM-L12-H384-uncased), which is small BERT-like model distilled from a larger one.

In [49]:
!pip install transformers -q

In [46]:
mostra_memoria()

Your runtime RAM in gb: 
 total 27.33
 available 25.74
 used 1.2
 free 11.14
 cached 14.55
 buffers 0.45


In [50]:
import random
import torch
import torch.nn.functional as F
import numpy as np

In [52]:
import transformers

In [53]:
mostra_memoria()

Your runtime RAM in gb: 
 total 27.33
 available 25.4
 used 1.54
 free 10.79
 cached 14.56
 buffers 0.45


## Fixando as seeds

In [47]:
def inicializa_seed(num_semente:int=123):
  """
  É recomendado reiniciar as seeds antes de inicializar o modelo, pois assim
  garantimos que os pesos vao ser sempre os mesmos.
  fontes de apoio: 
      http://nlp.seas.harvard.edu/2018/04/03/attention.html
      https://github.com/CyberZHG/torch-multi-head-attention/blob/master/torch_multi_head_attention/multi_head_attention.py#L15
  """
  random.seed(num_semente)
  np.random.seed(num_semente)
  torch.manual_seed(num_semente)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

  #torch.cuda.manual_seed(num_semente)
  #Cuda algorithms
  #torch.backends.cudnn.deterministic = True 

In [147]:
num_semente=123
inicializa_seed(num_semente)

## Definindo Hiperparâmetros iniciais

In [55]:
def inicia_hparam()->dict:
  # Inicialização dos parâmetros
  hparam = {}
  hparam["num_workers_dataloader"] = 1
  hparam["device"] = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  if torch.cuda.is_available(): print(torch. cuda. get_device_name(hparam["device"]))    
  return hparam

In [56]:
hparam=inicia_hparam()

Tesla T4


## Preparando para debug

https://zohaib.me/debugging-in-google-collab-notebook/

In [57]:
!pip install -Uqq ipdb
import ipdb
# %pdb off # desativa debug em exceção
# %pdb on  # ativa debug em exceção
# ipdb.set_trace(context=8)  para execução nesse ponto

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.3/793.3 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.8/385.8 KB 40.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython~=7.9.0, but you have ipython 8.11.0 which is incompatible.


In [ ]:
# %xmode Verbose 
# %xmode Plain
# colocar %debug após ocorrer exception

In [58]:
torch.autograd.set_detect_anomaly(True)

In [59]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [60]:
#transformers.utils.logging.set_verbosity_error()
transformers.utils.logging.set_verbosity_info()

# se usar datasets
#datasets.utils.logging.set_verbosity_error()
#datasets.utils.logging.set_verbosity_warning()


In [61]:
torch.set_printoptions(sci_mode=True) 

## Rastro (neptune.ai)

Gerado rastro da execução no Neptune (detalhes no artigo [Rastro-DM: Mineração de Dados com Rastro](https://revista.tcu.gov.br/ojs/index.php/RTCU/article/view/1664))


### Importação de libraries para Rastro

In [148]:
try:
  import neptune.new as neptune  
except ImportError:
  !pip install neptune-client
  raise Exception('Stopping RUNTIME! Please run again.')


/usr/local/lib/python3.9/dist-packages/neptune/internal/backends/hosted_client.py:48: NeptuneDeprecationWarning: The 'neptune-client' package has been deprecated and will be removed in the future. Install the 'neptune' package instead. For more, see https://docs.neptune.ai/setup/upgrading/
  from neptune.version import version as neptune_client_version
<ipython-input-148-faf515e8cb50>:2: NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
  import neptune.new as neptune


In [ ]:
try:
  from torchviz import make_dot 
except ImportError:
  !pip install torchviz
  from torchviz import make_dot 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4150 sha256=18dd865fc879cd9a8a79db282c5ecb09630e477bccc8e0b12fbc8c4087c5e1b4
  Stored in directory: /root/.cache/pip/wheels/04/38/f5/dc4f85c3909051823df49901e72015d2d750bd26b086480ec2
Successfully built torchviz


In [63]:
import getpass
import copy
import time
import re
import tempfile
import os


### Código Rastro

Busca implementar o rastro proposto em [Rastro-DM: Mineração de Dados com Rastro](https://revista.tcu.gov.br/ojs/index.php/RTCU/article/view/1664), autores Marcus Vinícius Borela de Castro e Remis Balaniuk, com o apoio da [solução Neptune](https://app.neptune.ai/)



In [64]:
def converte_optimizer_state_dict(parm_optimizer)-> dict:
  # return str(hparam['optimizer'])
  return parm_optimizer.state_dict()['param_groups'][0]

In [ ]:
def gera_tag_rastro_experiencia_treino(parm_aula:str, hparam:dict)->str:
  lista_tag = []
  lista_chaves = ['embed_dim', 'dim_feedforward', 'ind_activation_function',  'batch_size', 'learning_rate', 'weight_decay', 'amsgrad', 'decrease_factor_lr', 'max_examples', 'eval_every_steps']
  for chave in lista_chaves:
    tag = f"{chave} {hparam[chave]}"
    lista_tag.append(tag)
  return lista_tag

hparam['lista_tag_rastro_experiencia_treino'] =  gera_tag_rastro_experiencia_treino(parm_aula='aula7', hparam=hparam) 

In [65]:
class NeptuneRastroRun():
    se_geracao_rastro = True 
    neptune_project = ""
    tag_contexto_rastro = ""
    neptune_api_token = ""

    def __init__(self, parm_params:dict,  parm_lista_tag:list = None):
      # print(f"NeptuneRastroRun.init: se_geracao_rastro {self.__class__.se_geracao_rastro} parm_params `{parm_params} ")
      if self.__class__.se_geracao_rastro:      
        self.run_neptune = neptune.init(project=self.__class__.neptune_project, api_token=self.__class__.neptune_api_token, capture_hardware_metrics=True)
        self.run_neptune['sys/name'] = self.__class__.tag_contexto_rastro
        vparams = copy.deepcopy(parm_params)
        if "optimizer" in vparams:
          vparams["optimizer"] = converte_optimizer_state_dict(vparams["optimizer"])
        if 'criterion'  in vparams:
          vparams["criterion"] = str(vparams["criterion"])
        if 'scheduler'  in vparams:
          vparams["scheduler"] = str(type(vparams["scheduler"]))
        if 'device' in vparams:
          vparams['device'] = str(vparams["device"])
        self.device = vparams["device"]
        for tag in parm_lista_tag:
          self.run_neptune['sys/tags'].add(tag)
        self.run_neptune['parameters'] = vparams
        self.tmpDir = tempfile.mkdtemp()

    @property
    def run():
      return self.run_neptune

    @classmethod
    def ativa_geracao_rastro(cls):
     cls.se_geracao_rastro = True      

    @classmethod
    def def_contexto(cls):
     cls.se_geracao_rastro = True      

    @classmethod
    def desativa_geracao_rastro(cls):
     cls.se_geracao_rastro = False      

    @classmethod
    def retorna_status_geracao_rastro(cls):
     return cls.se_geracao_rastro      

    @classmethod
    def retorna_tag_contexto_rastro(cls):
     return cls.tag_contexto_rastro 

    @classmethod
    def inicia_contexto(cls, neptune_project, tag_contexto_rastro, neptune_api_token):
      assert '.' not in tag_contexto_rastro, "NeptuneRastroRun.init(): tag_contexto_rastro não pode possuir ponto, pois será usado para gravar nome de arquivo"      
      cls.neptune_api_token = neptune_api_token
      cls.tag_contexto_rastro = tag_contexto_rastro
      cls.neptune_project = neptune_project

    def salva_metrica(self, parm_metricas={}):
      #print(f"NeptuneRastroRun.salva_metrica: se_geracao_rastro {self.__class__.se_geracao_rastro} parm_metricas:{parm_metricas} ")
      if self.__class__.se_geracao_rastro:
        for metrica, valor in parm_metricas.items(): 
          self.run_neptune[metrica].log(valor)
 
    def gera_grafico_modelo(self, loader_train, model):
      if self.__class__.se_geracao_rastro: 
        # efetuar um forward 
        """
        se dataloader devolver x e y:
        """
        x_, y_ = next(iter(loader_train))
        x_ = x_.to(self.device)
        outputs = model(x_)
        """
        # se dataloader devolver dict:
        dados_ = next(iter(loader_train))
        outputs = model(dados_['x'].to(self.device))
        #outputs = model(x_['input_ids'].to(self.device), x_['attention_mask'].to(self.device))
        """
        nome_arquivo = os.path.join(self.tmpDir, "modelo "+ self.__class__.tag_contexto_rastro + time.strftime("%Y-%b-%d %H:%M:%S"))
        make_dot(outputs, params=dict(model.named_parameters()), show_attrs=True, show_saved=True).render(nome_arquivo, format="png")
        self.run_neptune["parameters/model_graph"].upload(nome_arquivo+'.png')
        self.run_neptune['parameters/model'] = re.sub('<bound method Module.state_dict of ', '',str(model.state_dict))      



    def stop(self):
      if self.__class__.se_geracao_rastro:         
        self.run_neptune.stop()


### Definindo parâmetros para o rastro



In [66]:
NeptuneRastroRun.inicia_contexto('marcusborela/IA386DD', 'Aula 10 - Modelo de linguagem treino auto-regressivo',   getpass.getpass('Informe NEPTUNE_API_TOKEN'))
#NeptuneRastroRun.desativa_geracao_rastro()

Informe NEPTUNE_API_TOKEN··········


## Preparing the data

First, we download the dataset:

In [67]:
path_data_train = path_data + '/train'
path_data_train

'/content/drive/MyDrive/treinamento/202301_IA368DD/collections/msmarco-passage/train'

In [80]:
path_data_train

'/content/drive/MyDrive/treinamento/202301_IA368DD/collections/msmarco-passage/train'

In [71]:
%%time
if not os.path.exists(path_data_train):
  os.makedirs(path_data_train)
  print(f'{path_data_train} pasta criada!')
else:
  print(f'{path_data_train} pasta já existia!')

/content/drive/MyDrive/treinamento/202301_IA368DD/collections/msmarco-passage/train pasta já existia!
CPU times: user 1.26 ms, sys: 0 ns, total: 1.26 ms
Wall time: 1.47 ms


In [81]:
if not os.path.exists(path_data_train+'/msmarco_triples.train.tiny.tsv'):
  !wget -P {path_data_train} https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv  
  print(f'arquivo baixado!')

/content/drive/MyDrive/treinamento/202301_IA368DD/collections/msmarco-passage/train: Scheme missing.
--2023-03-11 18:50:06--  https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 108.177.97.128, 108.177.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8076179 (7.7M) [text/tab-separated-values]
Saving to: ‘storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv’

storage.googleapis. 100%[===================>]   7.70M  --.-KB/s    in 0.09s   

2023-03-11 18:50:07 (87.8 MB/s) - ‘storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv’ saved [8076179/8076179]

FINISHED --2023-03-11 18:50:07--
Total wall clock time: 0.1s
Downloaded: 1 files, 7.7M in 0.09s (87.8 MB/s)
arquivo baixado!


In [86]:
!head {path_data_train+'/msmarco_triples.train.tiny.tsv'}

is a little caffeine ok during pregnancy	We donât know a lot about the effects of caffeine during pregnancy on you and your baby. So itâs best to limit the amount you get each day. If youâre pregnant, limit caffeine to 200 milligrams each day. This is about the amount in 1Â½ 8-ounce cups of coffee or one 12-ounce cup of coffee.	It is generally safe for pregnant women to eat chocolate because studies have shown to prove certain benefits of eating chocolate during pregnancy. However, pregnant women should ensure their caffeine intake is below 200 mg per day.
what fruit is native to australia	Passiflora herbertiana. A rare passion fruit native to Australia. Fruits are green-skinned, white fleshed, with an unknown edible rating. Some sources list the fruit as edible, sweet and tasty, while others list the fruits as being bitter and inedible.assiflora herbertiana. A rare passion fruit native to Australia. Fruits are green-skinned, white fleshed, with an unknown edible rating. Some sou

In [100]:
import pandas as pd

In [101]:
df = pd.read_csv(path_data_train+'/msmarco_triples.train.tiny.tsv', sep='\t', header=None)

In [104]:
df.columns = ['query', 'positive', 'negative']

In [103]:
df.head()

,query,positive passage,negative passage
0,is a little caffeine ok during pregnancy,We donât know a lot about the effects of caf...,It is generally safe for pregnant women to eat...
1,what fruit is native to australia,Passiflora herbertiana. A rare passion fruit n...,"The kola nut is the fruit of the kola tree, a ..."
2,how large is the canadian military,The Canadian Armed Forces. 1 The first large-...,The Canadian Physician Health Institute (CPHI)...
3,types of fruit trees,Cherry. Cherry trees are found throughout the ...,"The kola nut is the fruit of the kola tree, a ..."
4,how many calories a day are lost breastfeeding,"Not only is breastfeeding better for the baby,...","However, you still need some niacin each day; ..."


In [110]:
df.dtypes


query       object
positive    object
negative    object
dtype: object

Verificando correção do arquivo!

In [128]:
print(df.isnull().sum())

query       0
positive    0
negative    0
dtype: int64


Ajustando o texto no dataframe

In [123]:
df['query']=df['query'].astype(str)
df['positive']=df['positive'].astype(str)
df['negative']=df['negative'].astype(str)

In [125]:
df['query'] = df['query'].apply(lambda x: x.replace('â', '\''))
df['positive'] = df['positive'].apply(lambda x: x.replace('â', '\''))
df['negative'] = df['negative'].apply(lambda x: x.replace('â', '\''))


Verificando média de tamanho (em palavras, não em tokens)

In [174]:
df.applymap(len).mean()

query        34.225455
positive    353.499727
negative    340.203909
dtype: float64

In [127]:
df.shape

(11000, 3)

In [120]:
x = 'We donât know a lot about the effects'
#x = x.encode('utf-8').decode('unicode_escape')
x = x.replace('â', '\'')
print(x)

We don't know a lot about the effects


In [126]:
df.head()

,query,positive,negative
0,is a little caffeine ok during pregnancy,We don't know a lot about the effects of caffe...,It is generally safe for pregnant women to eat...
1,what fruit is native to australia,Passiflora herbertiana. A rare passion fruit n...,"The kola nut is the fruit of the kola tree, a ..."
2,how large is the canadian military,The Canadian Armed Forces. 1 The first large-...,The Canadian Physician Health Institute (CPHI)...
3,types of fruit trees,Cherry. Cherry trees are found throughout the ...,"The kola nut is the fruit of the kola tree, a ..."
4,how many calories a day are lost breastfeeding,"Not only is breastfeeding better for the baby,...","However, you still need some niacin each day; ..."


In [140]:
df_data = pd.DataFrame()

In [141]:
df_data['query']=df['query']
df_data['passage']=df['positive']
df_data['classe']=1


In [142]:
df_data = df_data.append(df[['query','negative']].rename(columns={'negative': 'passage'}).assign(classe=0), ignore_index=True)

<ipython-input-142-05336a35200b>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_data = df_data.append(df[['query','negative']].rename(columns={'negative': 'passage'}).assign(classe=0), ignore_index=True)


In [135]:
df[['query','negative']].assign(classe=0)

,query,negative,classe
0,is a little caffeine ok during pregnancy,It is generally safe for pregnant women to eat...,0
1,what fruit is native to australia,"The kola nut is the fruit of the kola tree, a ...",0
2,how large is the canadian military,The Canadian Physician Health Institute (CPHI)...,0
3,types of fruit trees,"The kola nut is the fruit of the kola tree, a ...",0
4,how many calories a day are lost breastfeeding,"However, you still need some niacin each day; ...",0
...,...,...,...
10995,where is maramal in skyrim,One type of character isn't necessarily better...,0
10996,which is a basic unit of a sugar molecule,According to Merriam-Webster and the Online Et...,0
10997,deloitte consulting or phone number,Virtual phone numbers are convenient for both ...,0
10998,"how many hours a day to work to get lunch in ,a",1 Levaquin can be taken at any time of the day...,0


In [143]:
df_data.shape

(22000, 3)

In [145]:
df_data.head(), df_data.tail()

(                                            query  \
 0        is a little caffeine ok during pregnancy   
 1               what fruit is native to australia   
 2              how large is the canadian military   
 3                            types of fruit trees   
 4  how many calories a day are lost breastfeeding   
 
                                              passage  classe  
 0  We don't know a lot about the effects of caffe...       1  
 1  Passiflora herbertiana. A rare passion fruit n...       1  
 2  The Canadian Armed Forces. 1  The first large-...       1  
 3  Cherry. Cherry trees are found throughout the ...       1  
 4  Not only is breastfeeding better for the baby,...       1  ,
                                                  query  \
 21995                       where is maramal in skyrim   
 21996        which is a basic unit of a sugar molecule   
 21997              deloitte consulting or phone number   
 21998  how many hours a day to work to get lunch in 

Dividindo o dataset em treino e validação de forma estratificada

In [146]:
from sklearn.model_selection import train_test_split

In [161]:
hparam['percent_test_size'] = .15 #@param [.10, .15, .20] {type:'raw')

In [163]:
X_train, X_valid, Y_train, Y_valid = train_test_split(df_data[['query','passage']], df_data['classe'], test_size=hparam['percent_test_size'], stratify=df_data['classe'], random_state=num_semente)

In [164]:
X_train.shape, type(X_valid), type(Y_valid), X_valid.shape, Y_valid.shape

((18700, 2),
 pandas.core.frame.DataFrame,
 pandas.core.series.Series,
 (3300, 2),
 (3300,))

In [165]:
print(Y_train.value_counts(), '\n', Y_valid.value_counts())

1    9350
0    9350
Name: classe, dtype: int64 
 1    1650
0    1650
Name: classe, dtype: int64


In [166]:
hparam['num_sentenca_train'] = X_train.shape[0]
hparam['num_sentenca_valid'] = X_valid.shape[0]

# visto que "max_position_embeddings": 512 em https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/blob/main/config.json
hparam['max_seq_length'] = 512 #@param [100, 512] {type:'raw'}


In [167]:
path_aula = '/content/drive/MyDrive/treinamento/202301_IA368DD/aula2/'
if not os.path.exists(path_aula):
  os.makedirs(path_aula)
  print(f'{path_aula} pasta criada!')
else: 
    print(f'{path_aula} pasta já existia!')
  

/content/drive/MyDrive/treinamento/202301_IA368DD/aula2/ pasta criada!


In [168]:
infixo_nome= 'rerank_max_seq_'+ str(hparam['max_seq_length'])+'_text_'
print(path_aula + infixo_nome)

/content/drive/MyDrive/treinamento/202301_IA368DD/aula2/rerank_max_seq_512_text_


### Computing input lengths

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = 'microsoft/MiniLM-L12-H384-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpmlnn14f3


Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

storing https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/ceb753d3f27a8c0d09184f35884666cda91b8ae610cd2a54d89793ac7663f1f9.13815020fd994b27db9974c0ce0ec4c47dfac6c8f11bf1a35a0a06d5b165665a
creating metadata file for /root/.cache/huggingface/transformers/ceb753d3f27a8c0d09184f35884666cda91b8ae610cd2a54d89793ac7663f1f9.13815020fd994b27db9974c0ce0ec4c47dfac6c8f11bf1a35a0a06d5b165665a
loading configuration file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ceb753d3f27a8c0d09184f35884666cda91b8ae610cd2a54d89793ac7663f1f9.13815020fd994b27db9974c0ce0ec4c47dfac6c8f11bf1a35a0a06d5b165665a
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_n

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

storing https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/49c302ee103bf6737d0877cfbd658563cf4bbc4b7914363ca419ce8a3d8a4c51.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
creating metadata file for /root/.cache/huggingface/transformers/49c302ee103bf6737d0877cfbd658563cf4bbc4b7914363ca419ce8a3d8a4c51.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp68qnbrsl


Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

storing https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/1e5909e4dfaa904617797ed35a6105a23daa56cbefca48fef329f772584699fb.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
creating metadata file for /root/.cache/huggingface/transformers/1e5909e4dfaa904617797ed35a6105a23daa56cbefca48fef329f772584699fb.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpeb77mm7a


Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

storing https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/29039dfe8c131360348e9f5ebecd464478cec7576c9af532b55ddcf9d4ec8d1e.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b
creating metadata file for /root/.cache/huggingface/transformers/29039dfe8c131360348e9f5ebecd464478cec7576c9af532b55ddcf9d4ec8d1e.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b
loading file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/49c302ee103bf6737d0877cfbd658563cf4bbc4b7914363ca419ce8a3d8a4c51.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/tokenizer.json from cache at None
loading file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/added_tokens.json from cache at None
loading file 

In [ ]:
mostra_memoria()

Your runtime RAM in gb: 
 total 27.33
 available 24.28
 used 2.65
 free 9.16
 cached 15.02
 buffers 0.49


In [182]:
tokenizer.pad_token_id,tokenizer.cls_token_id,tokenizer.sep_token_id

(0, 101, 102)

In [177]:
x='it is just a test!'

In [179]:
tokenizer(x)['input_ids']

[101, 2009, 2003, 2074, 1037, 3231, 999, 102]

In [180]:
df_tokenizado = df.applymap(lambda x: tokenizer(x)['input_ids'])

In [181]:
df_tokenizado.head()

,query,positive,negative
0,"[101, 2003, 1037, 2210, 24689, 7959, 3170, 792...","[101, 2057, 2123, 1005, 1056, 2113, 1037, 2843...","[101, 2009, 2003, 3227, 3647, 2005, 6875, 2308..."
1,"[101, 2054, 5909, 2003, 3128, 2000, 2660, 102]","[101, 3413, 10128, 10626, 2050, 7253, 11410, 1...","[101, 1996, 12849, 2721, 17490, 2003, 1996, 59..."
2,"[101, 2129, 2312, 2003, 1996, 3010, 2510, 102]","[101, 1996, 3010, 4273, 2749, 1012, 1015, 1996...","[101, 1996, 3010, 7522, 2740, 2820, 1006, 1813..."
3,"[101, 4127, 1997, 5909, 3628, 102]","[101, 9115, 1012, 9115, 3628, 2024, 2179, 2802...","[101, 1996, 12849, 2721, 17490, 2003, 1996, 59..."
4,"[101, 2129, 2116, 10250, 18909, 1037, 2154, 20...","[101, 2025, 2069, 2003, 7388, 7959, 17819, 248...","[101, 2174, 1010, 2017, 2145, 2342, 2070, 9152..."


In [188]:
print('max:', '\n', df_tokenizado.applymap(len).max(), '\n','mean:', '\n', df_tokenizado.applymap(len).mean(), '\n', 'std:', '\n', df_tokenizado.applymap(len).std())

max: 
 query        43
positive    280
negative    249
dtype: int64 
 mean: 
 query        9.104000
positive    80.878273
negative    77.519636
dtype: float64 
 std: 
 query        2.850345
positive    32.583392
negative    31.118472
dtype: float64


Logo: max_seq_length pode ser 323 (43 + 280)

#PAREI AQUI

In [172]:
from typing import List

In [173]:
class MyDataset():
    def __init__(self, text1: List[str], text2: List[str], classe:list[int], tokenizer, max_seq_length: int):
      """
      concatena tokenizer.cls_token_id, tokenizer.sep_token_id 
      gera sentencas x e y com max_seq_length
      como para a última posição de x não haveria label
        salva-se sentence_data com max_length + 1
        na hora de salvar
        x:sentence_data[0:max_length]
        y:sentence_data[1:max_length+1]
      """
      assert isinstance(text1,list), 'text1 deve ser do tipo list'
      assert isinstance(text1[0],str), 'text1[0] deve ser do tipo string'
      assert isinstance(text2,list), 'text2 deve ser do tipo list'
      assert isinstance(text2[0],str), 'text2[0] deve ser do tipo string'
      assert isinstance(classe,list), 'classe deve ser do tipo list'
      assert isinstance(classe[0],int), 'classe[0] deve ser do tipo int'
      assert isinstance(max_seq_length,int), 'max_seq_length deve ser do tipo int'
      assert max_seq_length > 100, 'max_seq_length deve ser maior do que 100'


### Loading the dataset

The IMDB dataset has 25,000 training + 25,000 test examples, but we will use only 1,000 for training, 1,000 for validation and another 1,000 for test.

In [ ]:
import os

max_train = 1000
max_valid = 1000
max_test = 1000

def load_texts(folder):
    texts = []
    for path in os.listdir(folder):
        with open(os.path.join(folder, path)) as f:
            texts.append(f.read())
    return texts

x_train_pos = load_texts('aclImdb/train/pos')
x_train_neg = load_texts('aclImdb/train/neg')
x_test_pos = load_texts('aclImdb/test/pos')
x_test_neg = load_texts('aclImdb/test/neg')

x_train = x_train_pos + x_train_neg
x_test = x_test_pos + x_test_neg
y_train = [True] * len(x_train_pos) + [False] * len(x_train_neg)
y_test = [True] * len(x_test_pos) + [False] * len(x_test_neg)

# Shuffle the training set first, and then split it into training and validation sets.
c = list(zip(x_train, y_train))
random.shuffle(c)
x_train, y_train = list(zip(*c))

# Now shuffle the test set.
c = list(zip(x_test, y_test))
random.shuffle(c)
x_test, y_test = list(zip(*c))

assert max_valid + max_train < len(x_train)

x_valid = list(x_train[-max_valid:])
y_valid = list(y_train[-max_valid:])
x_train = list(x_train[:max_train])
y_train = list(y_train[:max_train])
x_test = list(x_test[:max_test])
y_test = list(y_test[:max_test])

print(len(x_train), 'training examples.')
print(len(x_valid), 'valid examples.')
print(len(x_test), 'test examples.')

print('\nFirst 3 training examples:')
for x, y in zip(x_train[:3], y_train[:3]):
    print(y, x[:100])

print(100 * '-' + '\nFirst 3 validation examples:')
for x, y in zip(x_valid[:3], y_valid[:3]):
    print(y, x[:100])

print(100 * '-' + '\nFirst 3 test examples:')
for x, y in zip(x_test[:3], y_test[:3]):
    print(y, x[:100])

1000 training examples.
1000 valid examples.
1000 test examples.

First 3 training examples:
False This is a well made informative film in the vein of PBS Frontline. The problem is, Frontline already
False This movie was a major disappointment on direction, intellectual niveau, plot and in the way it deal
True Casting Jack Cassidy as Ken Frankin was sheer brilliance. Cassidy personified arrogance, confidence,
----------------------------------------------------------------------------------------------------
First 3 validation examples:
False Add to the list of caricatures: a Southern preacher and "congregation," a torch singer (Sophie Tucke
True I first read Pearl S Buck's splendid novel in my ninth grade history class, and I enjoyed every thri
True In the Old west there are always the men who live breathe violence and the women who hold their brea
----------------------------------------------------------------------------------------------------
First 3 test examples:
False A horrib

### Converting to Pytorch's Dataset/Dataloader

The classes Dataset + Dataloader allow us to efficiently feed data to train our model. More specifically, the class `Dataset` allows to randomly access an training example and the `DataLoader` class packs examples in batches.

In [ ]:
max_length = 300
x_train_tokenized = tokenizer(x_train, max_length=max_length, truncation=True)
x_valid_tokenized = tokenizer(x_valid, max_length=max_length, truncation=True)
x_test_tokenized = tokenizer(x_test, max_length=max_length, truncation=True)

In [ ]:
from torch.utils import data

class Dataset(data.Dataset):
    def __init__(self, examples, targets):
        self.examples = examples
        self.targets = targets
    
    def __len__(self):
        return len(self.examples['input_ids'])
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.examples['input_ids'][idx],
            'attention_mask': self.examples['attention_mask'][idx],
            'labels': int(self.targets[idx]),
        }

dataset_train = Dataset(x_train_tokenized, y_train)
assert len(dataset_train[0]['input_ids']) > 0
assert len(dataset_train[1]['attention_mask']) > 0
assert type(dataset_train[2]['labels']) == int

In [ ]:
# Convert examples to Pytorch's Dataset.
dataset_train = Dataset(x_train_tokenized, y_train)
dataset_valid = Dataset(x_valid_tokenized, y_valid)
dataset_test = Dataset(x_test_tokenized, y_test)

In [ ]:
from transformers import BatchEncoding

# This functions adds "pad" tokens to examples in the batch that are shorter than the largest one.
def collate_fn(batch):
  return BatchEncoding(tokenizer.pad(batch, return_tensors='pt'))

# Convert examples to Pytorch's DataLoader.
dataloader_train = data.DataLoader(dataset_train, batch_size=32, shuffle=True, collate_fn=collate_fn)
dataloader_valid = data.DataLoader(dataset_valid, batch_size=32, shuffle=False, collate_fn=collate_fn)
dataloader_test = data.DataLoader(dataset_test, batch_size=32, shuffle=False, collate_fn=collate_fn)

for batch in dataloader_train:
    assert batch['input_ids'].shape[0] <= dataloader_train.batch_size
    assert batch['input_ids'].shape[1] <= max_length
    break

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


## Finetuning

In [ ]:
# We first define the evaluation function to measure accuracy and loss

def evaluate(model, dataloader, set_name):
    losses = []
    correct = 0
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, mininterval=0.5, desc=set_name, disable=False):
            outputs = model(**batch.to(device))
            loss_val = outputs.loss
            losses.append(loss_val.cpu().item())
            preds = outputs.logits.argmax(dim=1)
            correct += (preds == batch['labels']).sum().item()

    print(f'{set_name} loss: {mean(losses):0.3f}; {set_name} accuracy: {correct / len(dataloader.dataset):0.3f}')

In [ ]:
from torch import nn
from torch import optim
from tqdm.auto import tqdm
from transformers import get_linear_schedule_with_warmup

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
print('Parameters', model.num_parameters())

epochs = 5
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
num_training_steps = epochs * len(dataloader_train)
# Warm up is important to stabilize training.
num_warmup_steps = int(num_training_steps * 0.1)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)

# First validation to check if evaluation code is working and accuracy is random as expected 
evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')

# Training loop
for epoch in tqdm(range(epochs), desc='Epochs'):
    model.train()
    train_losses = []
    for batch in tqdm(dataloader_train, mininterval=0.5, desc='Train', disable=False):
        optimizer.zero_grad()
        outputs = model(**batch.to(device))
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        train_losses.append(loss.cpu().item())

    print(f'Epoch: {epoch + 1} Training loss: {mean(train_losses):0.2f}')
    evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Parameters 33360770
Valid loss: 0.693; Valid accuracy: 0.506


Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

Train:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch: 1 Training loss: 0.67
Valid loss: 0.566; Valid accuracy: 0.721


...

Epoch: 3 Training loss: 0.32
Valid loss: 0.322; Valid accuracy: 0.871


Train:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch: 4 Training loss: 0.21
Valid loss: 0.316; Valid accuracy: 0.886


Train:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch: 5 Training loss: 0.16
Valid loss: 0.323; Valid accuracy: 0.885


## Evaluating on the test set

In [ ]:
evaluate(model=model, dataloader=dataloader_test, set_name='Test')

Test:   0%|          | 0/32 [00:00<?, ?it/s]

Test loss: 0.331; Test accuracy: 0.872


# Reranking a partir da saída do BM25